In [1]:
using JWAS,DataFrames,CSV, LinearAlgebra, DelimitedFiles, JWAS.misc, XSimPreProcess

## input file reading consideration for JWAS-SSBR :
## read io streams of 1) phenotype file 2) Obs_ID files for ID's to be tested
## io streams of 1) genotype file 2) map file 3) pedigree file to keep as iostream
## process map file using XSimPreProcess since it is generated from XSim (array format different from JWAS)
## save processed map file, then input the mapfile as io
## read mapfile as iosteam

phenofile  = "../../Processed-Data\\\\pheno_processed.txt" #Datasets.dataset
phenotypes1 = CSV.read(phenofile,delim = ',',header=true)
obs_ID = readdlm("../../Processed-Data\\Obs_ID.txt")
mapfile = "../../Raw-Data\\map_XSim_Raw.txt";
genofile   = "../../Processed-Data\\geno_processed.txt"#Datasets.dataset
pedfile = "../../Processed-Data\\pedigree_processed.txt"

## processing and reading mapfile as formatted iostream
new_map = process_map(mapfile)
processed_map = open("../../Processed-Data\\new_map_file_processed.txt", "w");
writedlm(processed_map, new_map)
close(processed_map)
newmapfile = "../../Processed-Data\\new_map_file_processed.txt"


## Run JWAS with 
pedigree=get_pedigree(pedfile,separator=",",header=true)
model_equation1  ="y = intercept";
R      = 1.0
model1 = build_model(model_equation1,R);
G3 =1.0
g = add_genotypes(model1,genofile,G3, separator=',', header=true);
outputEBV(model1, obs_ID);
outputMCMCsamples(model1)
out2=runMCMC(model1, phenotypes1, Pi=0.95,estimatePi=true,methods="BayesC",single_step_analysis=true,
             pedigree=pedigree, chain_length=100,output_samples_frequency=100,outputEBV=true, output_samples_file="../Analysis-Results\\MCMC_Samples");
keys(out2)


## WPPA 
mrkr_file = "../Analysis-Results\\MCMC_Samples_marker_effects_y.txt";
g = GWAS(mrkr_file, newmapfile, model1, header=true, window_size="1 MB", threshold=0.05)
g
writedlm("../Analysis-Results\\WPPA-results.txt", g)

The delimiter in pedigree_processed.txt is ','.


coding pedigree... 100%|████████████████████████████████| Time: 0:00:00
calculating inbreeding...  86%|██████████████████████   |  ETA: 0:00:00

Finished!


calculating inbreeding... 100%|█████████████████████████| Time: 0:00:01


The delimiter in geno_processed.txt is ','.
The header (marker IDs) is provided in geno_processed.txt.


┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = ip:0x0
└ @ Core :-1


2999 markers on 5880 individuals were added.

The prior for marker effects variance is calculated from the genetic variance and π.
The mean of the prior for the marker effects variance is: 0.013378


A Linear Mixed Model was build using model equations:

y = intercept

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1
ϵ               factor       random            8320
J               covariate    fixed                1

MCMC Information:

methods                                      BayesC
chain_length                                    100
burnin                                            0
estimatePi                                     true
estimateScale                                 false
starting_value                                false
printout_frequency                              101
output_samples_frequency                        100
constraint                                    false
missing_phenot

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:05


Compute the posterior probability of association of the genomic window that explains more than 0.05 of the total genetic variance
